# PART ONE

This data was sourced from kaggle. Zack Thoutt scraped the website called winemag.com after watching 'somm', a documentary on master sommelier(the ability to identify wines through blind tasting)
This data is available under the Creative commons,Attribution and Share-Alike license and I do not own this data.It is being used for educational purposes only https://www.kaggle.com/zynicide/wine-reviews

# Import libraries and dependencies

In [1]:
import requests
import json
import sys
sys.path.append('./')
from config import gkey

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.notebook_repr_html',True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Read in file and remove unwanted columns

In [3]:
wineData = pd.read_csv('winemag_data.csv')
wineData = wineData.drop(columns=['Unnamed: 0', 'region_2', 'designation'], axis=1)
wineData.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,nan,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.00,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.00,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.00,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",87,65.00,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


#We will go through the title column and extract the year the wine is produced. We see a pattern of the year included in most if not all of the tasters reviews. We will use the isdigit() method in python which identifies numerical digits in strings. We will append this to a new list and use this for our analysis later on.


In [4]:
#Extract year from Title to get year column
title_list = [title for title in wineData["title"]]

def hasNumbers(stringsList):
    return any(char.isdigit() for char in stringsList) 

counter = 0
years = []

for title in title_list:
    strings_list = []
    numbers_list = []
    years_list = []
    
    for ele in title.split(' '):
        strings_list.append(ele)
    if hasNumbers(strings_list) == True:
        for x in strings_list:
            if x.isdigit():
                numbers_list.append(int(x))
        years_list = [year for year in numbers_list if 1980 < year < 2020]
        #if the years list contains more than one element, append only the first one
        if len(years_list) >= 1:
            years.append(years_list[0]) 
        elif len(years_list) == 0:
            years.append("N/A")
    if hasNumbers(strings_list) == False:
        years.append("N/A")
    counter = counter + 1


In [5]:
#add the year list to the dataframe
wineData['year'] = years
wineData.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,year
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,nan,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.00,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.00,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.00,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this...",87,65.00,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


In [6]:
#rename null values as N/A to easily drop them
wineData = wineData.fillna('N/A')

# Use Google's api to append latitudinal and longitudinal points

In [7]:
#create a new column called Location
wineData['Location'] = wineData[['province','country']].apply(lambda x: ','.join(x), axis=1)
wineData.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,year,Location
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,N/A,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,"Sicily & Sardinia,Italy"
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.00,Douro,N/A,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,"Douro,Portugal"
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.00,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,"Oregon,US"
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.00,Michigan,Lake Michigan Shore,Alexander Peartree,N/A,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,"Michigan,US"
4,US,"Much like the regular bottling from 2012, this...",87,65.00,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,"Oregon,US"


In [8]:
#use the google location api to collect lat and lng points for plotting later
combined_location = []
lat_lng = []

for l in wineData['Location']:
    #append non dups only
    if l not in combined_location:
        combined_location.append(l)
        

for location in combined_location:
    target_location = location
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_location, gkey)
    
    response = requests.get(target_url).json()
    lat_lng.append(response)

In [9]:
lats = []
lngs = []

for index, point in enumerate(lat_lng): 
    try:
        lat = lat_lng[index]['results'][0]['geometry']['location']['lat']
        lats.append(lat)
        lng = lat_lng[index]['results'][0]['geometry']['location']['lng']
        lngs.append(lng)
        
    except:
        lats.append('N/A')
        lngs.append('N/A')

In [10]:
# Create a new df and added lat and lng values
dict1 = {'Location':combined_location,
               'Latitude': lats,
               'Longitude': lngs}

coord_data = pd.DataFrame(dict1)
coord_data

,Location,Latitude,Longitude
0,"Sicily & Sardinia,Italy",37.60,14.02
1,"Douro,Portugal",41.50,-5.51
2,"Oregon,US",43.80,-120.55
3,"Michigan,US",44.31,-85.60
4,"Northern Spain,Spain",42.82,-1.64
...,...,...,...
421,"Vin de Pays de Velvendo,Greece",40.25,22.07
422,"Landwein Rhein,Germany",49.23,8.39
423,"Lesbos,Greece",39.26,26.28
424,"Távora-Varosa,Portugal",41.04,-7.76


In [11]:
#merge the location data with the existing wine_data dataFrame 
wine_data = pd.merge(wineData, coord_data, on='Location', how='inner')
wine_data.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,year,Location,Latitude,Longitude
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,N/A,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,"Sicily & Sardinia,Italy",37.60,14.02
1,Italy,"Here's a bright, informal red that opens with ...",87,16.00,Sicily & Sardinia,Vittoria,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,"Sicily & Sardinia,Italy",37.60,14.02
2,Italy,This is dominated by oak and oak-driven aromas...,87,N/A,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Masseria Setteporte 2012 Rosso (Etna),Nerello Mascalese,Masseria Setteporte,2012,"Sicily & Sardinia,Italy",37.60,14.02
3,Italy,Delicate aromas recall white flower and citrus...,87,19.00,Sicily & Sardinia,Sicilia,Kerin O’Keefe,@kerinokeefe,Baglio di Pianetto 2007 Ficiligno White (Sicilia),White Blend,Baglio di Pianetto,2007,"Sicily & Sardinia,Italy",37.60,14.02
4,Italy,"Aromas of prune, blackcurrant, toast and oak c...",87,35.00,Sicily & Sardinia,Sicilia,Kerin O’Keefe,@kerinokeefe,Canicattì 2009 Aynat Nero d'Avola (Sicilia),Nero d'Avola,Canicattì,2009,"Sicily & Sardinia,Italy",37.60,14.02


In [12]:
# clean the column names  
wine_data.columns= ['Country', 'Title','Points','Price', 'Province','Region','Taster Name',

                    'Twitter Handle','Description','Variety','Winery','Year','Location','Latitude','Longitude']
print(len(wine_data))
wine_data.head()

129971


,Country,Title,Points,Price,Province,Region,Taster Name,Twitter Handle,Description,Variety,Winery,Year,Location,Latitude,Longitude
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,N/A,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,"Sicily & Sardinia,Italy",37.60,14.02
1,Italy,"Here's a bright, informal red that opens with ...",87,16.00,Sicily & Sardinia,Vittoria,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,"Sicily & Sardinia,Italy",37.60,14.02
2,Italy,This is dominated by oak and oak-driven aromas...,87,N/A,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Masseria Setteporte 2012 Rosso (Etna),Nerello Mascalese,Masseria Setteporte,2012,"Sicily & Sardinia,Italy",37.60,14.02
3,Italy,Delicate aromas recall white flower and citrus...,87,19.00,Sicily & Sardinia,Sicilia,Kerin O’Keefe,@kerinokeefe,Baglio di Pianetto 2007 Ficiligno White (Sicilia),White Blend,Baglio di Pianetto,2007,"Sicily & Sardinia,Italy",37.60,14.02
4,Italy,"Aromas of prune, blackcurrant, toast and oak c...",87,35.00,Sicily & Sardinia,Sicilia,Kerin O’Keefe,@kerinokeefe,Canicattì 2009 Aynat Nero d'Avola (Sicilia),Nero d'Avola,Canicattì,2009,"Sicily & Sardinia,Italy",37.60,14.02


In [13]:
#Remove rows from columns price, province,points, lat and lng with N/A values
wine_data = wine_data.loc[wine_data['Price'] != 'N/A']
wine_data = wine_data.loc[wine_data['Province'] != 'N/A']  
wine_data = wine_data.loc[wine_data['Year'] != 'N/A']
wine_data = wine_data.loc[wine_data['Latitude'] !='N/A']
wine_data = wine_data.loc[wine_data['Longitude'] !='N/A']

wine_data.shape

(114589, 15)

In [14]:
# Get the unique names in wine variety to group wine by category
wine_data['Variety'].unique()

array(['Frappato', 'White Blend', "Nero d'Avola", 'Red Blend',
       'Cabernet Sauvignon', 'Catarratto', 'Inzolia', 'Monica', 'Grillo',
       'Nerello Mascalese', 'Cannonau', 'Carricante', 'Rosato',
       'Pinot Noir', 'Syrah', 'Insolia', 'Chardonnay', 'Zibibbo', 'Fiano',
       'Rosé', 'Vermentino', 'Sauvignon', 'Albanello', 'Perricone',
       'Carignano', 'Torbato', 'Cabernet Franc', 'Merlot', 'Moscato',
       'Sauvignon Blanc', 'Pinot Nero', 'Nerello Cappuccio', 'Malvasia',
       'Grecanico', 'Pinot Grigio', 'Nuragus', 'Sangiovese', 'Shiraz',
       'Bovale', 'Mondeuse', 'Tannat', 'Cabernet Blend', 'Viognier',
       'Nasco', 'Alicante', 'Müller-Thurgau', 'Moscato di Noto',
       'Francisa', 'Cabernet Sauvignon-Shiraz', 'Portuguese Red',
       'Portuguese White', 'Sousão', 'Touriga Nacional',
       'Portuguese Sparkling', 'Tinta Barroca', 'Tinta Roriz',
       'Touriga Franca', 'Portuguese Rosé', 'Touriga Nacional Blend',
       'Códega do Larinho', 'Viosinho', 'Tinta Franc

In [15]:
# Add a new column called Category to df
wine_data['Category'] = ''


In [16]:
#Assign labels Red to red wines and White to white wines 
wine_data["Category"] = np.where(wine_data["Variety"].str.contains(r'Albanello|Blanc|Gris|Grillo|Blanche|White|Chardonnay|Moscato|Musca| \
           Narince|Riesling|Catarratto|Carricante|Bianco|Semillon|Viognier|Cortese|Gewürztraminer|Fiano|Inzolia|Insolia|Zibibbo|Champagne| \
           Kisi|Sparkling|Tamianka|Mtsvane|Orangetraube|Kangoun|Moschofilero|Prosecco|Liebfraumilch|Hondarrabi|Macabeo|Tamjanika|Xynisteri|Zilavka| \
           |Zlahtina|Athiri|Roussanne|Torbato', case=False), "White", "Red")
wine_data.head()

,Country,Title,Points,Price,Province,Region,Taster Name,Twitter Handle,Description,Variety,Winery,Year,Location,Latitude,Longitude,Category
1,Italy,"Here's a bright, informal red that opens with ...",87,16.00,Sicily & Sardinia,Vittoria,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,"Sicily & Sardinia,Italy",37.60,14.02,Red
3,Italy,Delicate aromas recall white flower and citrus...,87,19.00,Sicily & Sardinia,Sicilia,Kerin O’Keefe,@kerinokeefe,Baglio di Pianetto 2007 Ficiligno White (Sicilia),White Blend,Baglio di Pianetto,2007,"Sicily & Sardinia,Italy",37.60,14.02,White
4,Italy,"Aromas of prune, blackcurrant, toast and oak c...",87,35.00,Sicily & Sardinia,Sicilia,Kerin O’Keefe,@kerinokeefe,Canicattì 2009 Aynat Nero d'Avola (Sicilia),Nero d'Avola,Canicattì,2009,"Sicily & Sardinia,Italy",37.60,14.02,Red
5,Italy,Pretty aromas of yellow flower and stone fruit...,87,13.00,Sicily & Sardinia,Terre Siciliane,Kerin O’Keefe,@kerinokeefe,Stemmari 2013 Dalila White (Terre Siciliane),White Blend,Stemmari,2013,"Sicily & Sardinia,Italy",37.60,14.02,White
6,Italy,"Aromas recall ripe dark berry, toast and a whi...",87,10.00,Sicily & Sardinia,Terre Siciliane,Kerin O’Keefe,@kerinokeefe,Stemmari 2013 Nero d'Avola (Terre Siciliane),Nero d'Avola,Stemmari,2013,"Sicily & Sardinia,Italy",37.60,14.02,Red


# Remove Countries that the wine tasters mentioned less than 50 times

In [23]:
#Replace the name US with United States
wine_data["Country"] = wine_data["Country"].replace(
    {"US": "United States"})

In [24]:
# We want to see the number of times each country is mentioned 
wine_data['Country'].value_counts()

United States    53351
France           15892
Italy            14966
Spain             6062
Portugal          4546
Chile             4391
Argentina         3714
Austria           2656
Australia         2225
Germany           1827
New Zealand       1368
South Africa      1278
Israel             471
Greece             425
Canada             253
Hungary            142
Bulgaria           141
Romania            111
Uruguay            108
Georgia             81
Slovenia            78
Turkey              74
Mexico              70
Croatia             57
Moldova             57
England             55
Name: Country, dtype: int64

In [25]:
#remove countries with less than 50 mentions
from collections import Counter

counts = Counter(wine_data.Country)
wine_data = wine_data[wine_data.Country.isin([num for num in counts if counts[num] > 50])]
wine_data = wine_data.reset_index()

In [26]:
wine_data = wine_data.drop('index', axis=1)

In [27]:
wine_data.head()

,Country,Title,Points,Price,Province,Region,Taster Name,Twitter Handle,Description,Variety,Winery,Year,Location,Latitude,Longitude,Category
0,Italy,"Here's a bright, informal red that opens with ...",87,16.00,Sicily & Sardinia,Vittoria,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,"Sicily & Sardinia,Italy",37.60,14.02,Red
1,Italy,Delicate aromas recall white flower and citrus...,87,19.00,Sicily & Sardinia,Sicilia,Kerin O’Keefe,@kerinokeefe,Baglio di Pianetto 2007 Ficiligno White (Sicilia),White Blend,Baglio di Pianetto,2007,"Sicily & Sardinia,Italy",37.60,14.02,White
2,Italy,"Aromas of prune, blackcurrant, toast and oak c...",87,35.00,Sicily & Sardinia,Sicilia,Kerin O’Keefe,@kerinokeefe,Canicattì 2009 Aynat Nero d'Avola (Sicilia),Nero d'Avola,Canicattì,2009,"Sicily & Sardinia,Italy",37.60,14.02,Red
3,Italy,Pretty aromas of yellow flower and stone fruit...,87,13.00,Sicily & Sardinia,Terre Siciliane,Kerin O’Keefe,@kerinokeefe,Stemmari 2013 Dalila White (Terre Siciliane),White Blend,Stemmari,2013,"Sicily & Sardinia,Italy",37.60,14.02,White
4,Italy,"Aromas recall ripe dark berry, toast and a whi...",87,10.00,Sicily & Sardinia,Terre Siciliane,Kerin O’Keefe,@kerinokeefe,Stemmari 2013 Nero d'Avola (Terre Siciliane),Nero d'Avola,Stemmari,2013,"Sicily & Sardinia,Italy",37.60,14.02,Red


In [28]:
#see the number of unique values in relevant columns and a summary dataframe
country = len(wine_data['Country'].unique())
title = len(wine_data['Title'].unique())
winery = len(wine_data['Winery'].unique())
variety = len(wine_data['Variety'].unique())
price = len(wine_data['Price'].unique())
province=len(wine_data['Province'].unique())
year = len(wine_data['Year'].unique())
grade = len(wine_data['Points'].unique())



df = pd.DataFrame({'Number of Titles': [title],
                   'Number of Countries': [country],
                   'Number of Wineries': [winery],
                   'Variety count': [variety],
                   'Prices count': [price],
                   'Number of provinces': [province],
                   'Year count' : [year],
                   'Wine grades' :[grade]
                  })
                        
df

,Number of Titles,Number of Countries,Number of Wineries,Variety count,Prices count,Number of provinces,Year count,Wine grades
0,105494,26,15105,629,382,355,36,21


# Export as csv file and load to sqlite database

In [29]:
# save completed and cleaned df and export to csv
wine_data.to_csv('Wine_data.csv')

In [30]:
# export to sqlite database
from sqlalchemy import create_engine
import sqlite3

engine = create_engine('sqlite:///wine_reviews.db', echo=True)
conn = engine.connect()

2020-11-08 20:46:13,736 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-11-08 20:46:13,738 INFO sqlalchemy.engine.base.Engine ()
2020-11-08 20:46:13,740 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-11-08 20:46:13,741 INFO sqlalchemy.engine.base.Engine ()


In [31]:
# create table and load dataframe to sqlite db
sqlite_table = 'wine'
wine_data.to_sql(sqlite_table, conn, if_exists='replace')


2020-11-08 20:46:15,912 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("wine")
2020-11-08 20:46:15,914 INFO sqlalchemy.engine.base.Engine ()
2020-11-08 20:46:15,917 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("wine")
2020-11-08 20:46:15,919 INFO sqlalchemy.engine.base.Engine ()
2020-11-08 20:46:15,921 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-11-08 20:46:15,923 INFO sqlalchemy.engine.base.Engine ()
2020-11-08 20:46:16,219 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("wine")
2020-11-08 20:46:16,220 INFO sqlalchemy.engine.base.Engine ()
2020-11-08 20:46:16,223 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'wine' AND type = 'table'
2020-11-08 20:46:16,224 INFO sqlalchemy.engine.base.Engine ()
2020-11-08 20:46:16,226 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("wine")
2020-11-08 20:46:16,227 INFO sq